In [2]:
from PIL import Image
import pandas as pd
import numpy as np
import random 
import matplotlib.pyplot as plt 
import tensorflow
import imageio
from scipy import signal
import os


In [5]:
all_image_array=[]
all_image_array_gauss=[]

files_dir = r'C:\Users\roumba\Documents\Software\deep-events\Videos\done\220915_mtstaygold_cos7_ZEIS_1'
images_dir = 'deep_events1b'
for image_index in range(0,28):
    file_name = f'image_{image_index}.tiff'
    file_name_gauss = f'image_{image_index}gauss.tiff'
    joined_path = os.path.join(files_dir, images_dir, file_name)
    joined_path_gauss = os.path.join(files_dir, images_dir, file_name_gauss)
    img = Image.open(str(joined_path).strip())
    image_array = np.array(img)
    img_gauss = Image.open(joined_path_gauss)
    image_array_gauss = np.array(img_gauss)
    all_image_array.append(image_array)
    all_image_array_gauss.append(image_array_gauss)
